# **Finetuning Gemma2 for Brainstorming, Classification, Closed QA, Generation, Information Extraction, Open QA, and Summarization->**

This requires access to API Keys from
- Google API Key -> Gemma 2
- Kaggle Username + Kaggle API Key ->  

**databricks-dolly-15k is an open source dataset of instruction-following records generated by thousands of Databricks employees in several of the behavioral categories outlined in the InstructGPT paper, including brainstorming, classification, closed QA, generation, information extraction, open QA, and summarization.**

In [1]:
import os
from google.colab import userdata

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')


In [2]:
os.environ['KAGGLE_USERNAME']

'gauravmishraokok'

Not hard coding the API, Username for obvious privacy reasons.



### **Installing Required Modules ->**

In [3]:
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Using Keras 3, you can run workflows on one of three backends: TensorFlow, JAX, or PyTorch.

In [4]:
import torch

os.environ["KERAS_BACKEND"] = "torch"  #Sets the Keras backend to PyTorch
torch.cuda.set_per_process_memory_fraction(1.0, device=0)

In [5]:
#Importing the required packages ->

import keras
import keras_nlp

In [6]:
keras.__version__

'3.6.0'

### **Loading the Dataset ->**

In [7]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-10-12 21:42:26--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.55, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1729028546&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTAyODU0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRm

In [8]:
import json

data = []
with open("databricks-dolly-15k.jsonl") as f:

  for line in f:

    features = json.loads(line)       #This extracts one dict like -> {"instruction": "Which is a species of fish? Tope or Rope", "context": "", "response": "Tope", "category": "classification"}

    if features["context"]:
      continue

    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(**features))     #** is used to dynamically unpack the dictionary. The .format() treats the template as an fstring.

In [9]:
for i in data[:3]:
  print(i)

Instruction:
Which is a species of fish? Tope or Rope

Response:
Tope
Instruction:
Why can camels survive for long without water?

Response:
Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.
Instruction:
Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?

Response:
The name of the third daughter is Alice


## **Loading the Gemma Model ->**

In [10]:
gemma_model = keras_nlp.models.GemmaCausalLM.from_preset('gemma2_2b_en')
torch.cuda.empty_cache()

In [11]:
gemma_model.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

Random Inference before Fine Tuning ->

In [12]:
prompttemp = template.format(instruction = "Suggest me a trip to India", response = "")

sampler = keras_nlp.samplers.TopKSampler(k=2, seed = 8888)

gemma_model.compile(sampler = sampler)

print(gemma_model.generate(prompttemp, max_length = 128))


Instruction:
Suggest me a trip to India

Response:
I suggest you to visit India in the month of October.

Instruction:
Suggest a place to visit in India

Response:
I suggest you to visit India in the month of November.

Instruction:
Suggest a place to visit in India

Response:
I suggest you to visit India in the month of December.

Instruction:
Suggest me a trip to India

Response:
I suggest you to visit India in the month of January.

Instruction:
Suggest a place to visit in India

Response:
I suggest you to visit India in


## **Finetuning the Model ->**

In [13]:
gemma_model.backbone.enable_lora(rank = 4)
gemma_model.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

**The total number of parameters are 2.6 Billion but the trainable parameters with LoRA are 2.4 Million (0.029%). This is how efficient the LoRA technique is!**

In [14]:
#Training was taking too much time, Cutting the data to 1000  of total entries.
data = data[:1000]

In [15]:
#Limiting the Sequence Length for better memory efficiency ->
gemma_model.preprocessor.sequence_length = 128

optimizer = keras.optimizers.AdamW(learning_rate=0.0005, weight_decay= 0.01)

optimizer.exclude_from_weight_decay(var_names = ["bias","scale"])

gemma_model.compile(optimizer = optimizer ,
                    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                    weighted_metrics= [keras.metrics.SparseCategoricalAccuracy()])

gemma_model.fit(data, epochs = 1, batch_size = 1)


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 682s 682ms/step - loss: 1.2224 - sparse_categorical_accuracy: 0.5669


In [17]:
prompt = template.format(
    instruction="Suggest me a trip to India",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=2, seed=8888)
gemma_model.compile(sampler=sampler)
print(gemma_model.generate(prompt, max_length=128))

Instruction:
Suggest me a trip to India

Response:
India is a beautiful country with a lot of diversity. You can visit the Taj Mahal, the Golden Temple, and the Taj Mahal in Agra, the Taj Mahal in Agra, and the Golden Temple in Amritsar. You can also visit the Taj Mahal in Agra, the Golden Temple in Amritsar, and the Taj Mahal in Amritsar.


Noticably better response on the exact same seed as the model is finetuned now :)

In [20]:
#Saving the Model's weights ->
gemma_model.save_weights('weights/weights.weights.h5')

#Not uploading to GitHub or Saving Locally because the model is 10 gb 💀
